In [1]:
import python_utils as utils
import sys
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
from matplotlib.gridspec import GridSpec
import matplotlib.colors as colors

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 13


In [2]:
#messo n_parseg = 150 e sembra avere senso lo spettrogramma anche se non sono 100 come nell'articolo
parseg = 250
neglect_t = 3000
def mask(f, lower_bound=12, upper_bound=30):
    return (lower_bound <= f) & (f <= upper_bound)

# Funzione per il calcolo della media mobile migliorata
def moving_average(data, window_size):
    moving_avg = np.empty(len(data))
    moving_avg[:] = np.NaN  # Inizializza con NaN
    half_window = window_size // 2
    for i in range(half_window, len(data) - half_window):
        moving_avg[i] = np.mean(data[i - half_window:i + half_window + 1])
    return moving_avg

# constant values

In [3]:
function = "flat_begin"
subnets = "STN"
sigm_par = float('inf')

new_function = re.sub(r"[^0-9]", "", function) #lascia solo numero funzione

In [4]:
dd_values = np.linspace(0.6, 1.1, 6)

In [5]:
folder_names = [f"flat_begin_0.00_0.00000_0.00_{dd:.3f}_1.00_1.00_1.6" for dd in dd_values]

In [6]:
# Prima, determiniamo i valori minimo e massimo globali per Sxx_lim
min_Sxx = np.inf
max_Sxx = -np.inf

for dd in dd_values:
    folder = f"flat_begin_0.00_0.00000_0.00_{dd:.3f}_1.00_1.00_1.6"
    file_path = f"./build2/output/n1/{folder}/ext_rateD2.txt"
    data3 = np.loadtxt(file_path).T
    
    # spikesim
    s3 = utils.SpikeSim(f"./build2/output/n1/{folder}", 'new_sim_parallel.yaml', neglect_t, np.max(data3[0])-neglect_t)
    
    output = s3.periodogram(pop=subnets, res=1., N_parseg=parseg)
    f = output[0]
    t = output[1]
    Sxx = output[2]
    data3[0] = data3[0]/1000
    data3[1] = data3[1]/1.083

    masked_indices = mask(output[0])  # Utilizza 'mask' per selezionare gli indici
    Sxx_lim = Sxx[masked_indices, :]
    min_Sxx = min(min_Sxx, np.min(Sxx_lim))
    max_Sxx = max(max_Sxx, np.max(Sxx_lim))

nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125


In [7]:
%matplotlib widget

# Ora, usa min_Sxx e max_Sxx per settare la scala comune nei subplot
fig, axs = plt.subplots(2, 3, figsize=(16, 8))
fig.subplots_adjust(hspace=0.4, wspace=0.4)
axs = axs.flatten()



for i, dd in enumerate(dd_values):
    folder = f"flat_begin_0.00_0.00000_0.00_{dd:.3f}_1.00_1.00_1.6"
    file_path = f"./build2/output/n1/{folder}/ext_rateD2.txt"
    data3 = np.loadtxt(file_path).T
    
    # spikesim
    s3 = utils.SpikeSim(f"./build2/output/n1/{folder}", 'new_sim_parallel.yaml', neglect_t, np.max(data3[0])-neglect_t)
    
    output = s3.periodogram(pop=subnets, res=1., N_parseg=parseg)
    f = output[0]
    t = output[1]
    Sxx = output[2]
    data3[0] = data3[0]/1000
    data3[1] = data3[1]/1.083

    # Assuming 'mask', 'Sxx', 'f', and 't' are defined in your analysis
    masked_indices = mask(f)  # Make sure 'mask' function is defined in 'utils'
    Sxx_lim = Sxx[masked_indices, :]
    f_lim = f[masked_indices]

    # Utilizza LogNorm per la scala dei colori in scala logaritmica
    norm = colors.LogNorm(vmin=np.max([min_Sxx, 1e-5]), vmax=max_Sxx)  # Imposta la normalizzazione logaritmica

    contour = axs[i].contourf(t, f_lim, Sxx_lim, norm=norm)
    axs[i].set_title(f"ξ = {dd:.1f}")
    axs[i].set_xlabel('Time [s]')
    
    if i % 3 == 0:
        axs[i].set_ylabel('Frequency [Hz]')
    
    # Aggiungi la colorbar con la scala logaritmica
    cbar = fig.colorbar(contour, ax=axs[i], orientation='vertical', label='PSD [u.a.]', fraction=0.046, pad=0.04)
    cbar.set_label('PSD [u.a.]', rotation=270, labelpad=15)

for ax in axs[len(dd_values):]:  # Nascondi gli assi inutilizzati
    ax.axis('off')

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125


# FLAT 0.85

In [8]:
Dd = "0.8500"
function = "flat"
subnets = "STN"
sigm_par = float('inf')

new_Dd = Dd.replace('.','_')
new_function = re.sub(r"[^0-9]", "", function) #lascia solo numero funzione

In [9]:
#check if periodogram folder exists and if not create one
periodogram_path = './periodograms/' + new_Dd + '_' + subnets + '_' + function
if not os.path.exists(periodogram_path):
    os.makedirs(periodogram_path)    

In [10]:
#funzione Dd
data1 = np.loadtxt("./build/output/n1/" + function + "_0.00_0.00000_0.00_" + Dd + "_1.00_1.00_1.6/0/ext_rateD2.txt").T
#spikesim
s1 = utils.SpikeSim("./build/output/n1/" + function + "_0.00_0.00000_0.00_" + Dd + "_1.00_1.00_1.6/0", 'new_sim_parallel.yaml', neglect_t, np.max(data1[0]))

In [11]:
data = data1
output=s1.periodogram(pop=subnets, res=1., N_parseg=parseg, save_img=periodogram_path + "/periodogram.png")
f = output[0]
t = output[1]
Sxx = output[2]
data[0] = data[0]/1000
data[1] = data[1]/1.083

nparseg = 250	noverlap=125


In [12]:
%matplotlib widget

# Applicazione della maschera ai dati
mask_f = mask(f)  # Assicurati che mask sia definita altrove nel tuo codice
Sxx_lim = Sxx[mask_f, :]
f_lim = f[mask_f]

# Impostazione del layout del grafico
fig = plt.figure(figsize=(10, 8))
gs = GridSpec(2, 2, width_ratios=[1, 0.05], height_ratios=[1, 1], hspace=0.3)

# Assegnazione degli assi
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0])
cax = fig.add_subplot(gs[0, 1])  # Modificato per abbracciare entrambe le righe

# Primo plot: contour plot per PSD
norm = colors.LogNorm(vmin=np.max([min_Sxx, 1e-5]), vmax=max_Sxx)  # Definizione della normalizzazione logaritmica
contour = ax1.contourf(t, f_lim, Sxx_lim, norm=norm)
ax1.set_title("STN PSD in the β Range ξ = 0.85")
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Frequency [Hz]')
ax1.text(0.01, 0.95, '(a)', transform=ax1.transAxes, fontsize=16, fontweight='bold', va='top')

# Aggiunta della colorbar con normalizzazione logaritmica
plt.colorbar(contour, cax=cax, orientation='vertical', label='PSD [u.a.]')

# Calcolo della media per ogni istante temporale
pow_t = [np.mean(Sxx_lim[:, i]) for i in range(len(t))]

# Calcolo della media mobile senza distorsione agli estremi
moving_avg = moving_average(np.array(pow_t), window_size=10)  # Assicurati che 'moving_average' sia definita

# Secondo plot: serie temporale dell'intensità media (Mean β PSD)
ax2.plot(t, pow_t, label='Mean β PSD')
mean_value = np.mean(pow_t)
ax2.plot([t[0], t[-1]], [mean_value, mean_value], color='red', label='Mean')
ax2.plot(t, moving_avg, label=f'Moving Average (10 points)', color='orange')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Mean β PSD [u.a.]')
ax2.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax2.set_title("Average PSD in the β Range")
ax2.text(0.01, 0.95, '(b)', transform=ax2.transAxes, fontsize=16, fontweight='bold', va='top')

# Impostazione dei limiti dell'asse x per corrispondere all'intervallo dei tuoi dati
ax2.set_xlim([t[0], t[-1]])

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# FLAT 0.95

In [13]:
Dd = "0.9500"
function = "flat"
subnets = "STN"
sigm_par = float('inf')

new_Dd = Dd.replace('.','_')
new_function = re.sub(r"[^0-9]", "", function) #lascia solo numero funzione

In [14]:
#check if periodogram folder exists and if not create one
periodogram_path = './periodograms/' + new_Dd + '_' + subnets + '_' + function
if not os.path.exists(periodogram_path):
    os.makedirs(periodogram_path)    

In [15]:
#funzione Dd
data2 = np.loadtxt("./build/output/n1/" + function + "_0.00_0.00000_0.00_" + Dd + "_1.00_1.00_1.6/0/ext_rateD2.txt").T
#spikesim
s2 = utils.SpikeSim("./build/output/n1/" + function + "_0.00_0.00000_0.00_" + Dd + "_1.00_1.00_1.6/0", 'new_sim_parallel.yaml', neglect_t, np.max(data2[0])-neglect_t)

In [16]:
data = data2
output=s2.periodogram(pop=subnets, res=1., N_parseg=parseg, save_img=periodogram_path + "/periodogram.png")
f = output[0]
t = output[1]
Sxx = output[2]
data[0] = data[0]/1000
data[1] = data[1]/1.083


nparseg = 250	noverlap=125


In [17]:
%matplotlib widget

# Applicazione della maschera ai dati
mask_f = mask(f)  # Assicurati che mask sia definita altrove nel tuo codice
Sxx_lim = Sxx[mask_f, :]
f_lim = f[mask_f]

# Impostazione del layout del grafico
fig = plt.figure(figsize=(10, 8))
gs = GridSpec(2, 2, width_ratios=[1, 0.05], height_ratios=[1, 1], hspace=0.3)

# Assegnazione degli assi
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0])
cax = fig.add_subplot(gs[0, 1])  # Modificato per abbracciare entrambe le righe

# Primo plot: contour plot per PSD
norm = colors.LogNorm(vmin=np.max([min_Sxx, 1e-5]), vmax=max_Sxx)  # Definizione della normalizzazione logaritmica
contour = ax1.contourf(t, f_lim, Sxx_lim, norm=norm)
ax1.set_title("STN PSD in the β Range ξ = 0.95")
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Frequency [Hz]')
ax1.text(0.01, 0.95, '(c)', transform=ax1.transAxes, fontsize=16, fontweight='bold', va='top')

# Aggiunta della colorbar con normalizzazione logaritmica
plt.colorbar(contour, cax=cax, orientation='vertical', label='PSD [u.a.]')

# Calcolo della media per ogni istante temporale
pow_t = [np.mean(Sxx_lim[:, i]) for i in range(len(t))]

# Calcolo della media mobile senza distorsione agli estremi
moving_avg = moving_average(np.array(pow_t), window_size=10)  # Assicurati che 'moving_average' sia definita

# Secondo plot: serie temporale dell'intensità media (Mean β PSD)
ax2.plot(t, pow_t, label='Mean β PSD')
mean_value = np.mean(pow_t)
ax2.plot([t[0], t[-1]], [mean_value, mean_value], color='red', label='Mean')
ax2.plot(t, moving_avg, label=f'Moving Average (10 points)', color='orange')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Mean β PSD [u.a.]')
ax2.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax2.set_title("Average PSD in the β Range")
ax2.text(0.01, 0.95, '(d)', transform=ax2.transAxes, fontsize=16, fontweight='bold', va='top')

# Impostazione dei limiti dell'asse x per corrispondere all'intervallo dei tuoi dati
ax2.set_xlim([t[0], t[-1]])

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## immagine DD

In [3]:
Dd_values = []
current_value = 0.850
while current_value <= 0.950:
    Dd_values.append(round(current_value, 4))
    current_value += 0.0025

print(Dd_values)

[0.85, 0.8525, 0.855, 0.8575, 0.86, 0.8625, 0.865, 0.8675, 0.87, 0.8725, 0.875, 0.8775, 0.88, 0.8825, 0.885, 0.8875, 0.89, 0.8925, 0.895, 0.8975, 0.9, 0.9025, 0.905, 0.9075, 0.91, 0.9125, 0.915, 0.9175, 0.92, 0.9225, 0.925, 0.9275, 0.93, 0.9325, 0.935, 0.9375, 0.94, 0.9425, 0.945, 0.9475, 0.95]


In [4]:
pow_t_mean_all = np.loadtxt('./build/output/n1/new_pow_t_mean' + str(parseg) + '.txt', delimiter='\t', skiprows=1)


In [5]:
int_all = np.mean(pow_t_mean_all, axis=0)
std_int = np.std(pow_t_mean_all, axis=0)


In [6]:
%matplotlib widget
y = 10 * (np.array(Dd_values) - 0.85)
# Creare un grafico dei valori nel tempo
# Finestra mobile (rolling window) per il calcolo della moving average
window_size = 1  # Sostituisci con la dimensione della finestra desiderata
moving_avg = np.convolve(int_all, np.ones(window_size)/window_size, mode='same')

# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize = (9,5))

ax1.set_yscale('log')

# Aggiungi la moving average al grafico di ax1
ax1.plot(y, moving_avg, color='blue')

# Aggiungi il terzo set di dati con errori associati al grafico esistente
ax1.errorbar(y, int_all, yerr=std_int, fmt='o')#, label='simulazioni a Dd costante')

# Imposta altre personalizzazioni per il tuo grafico, se necessario
plt.xlabel('y Dopamine depletion [u.a.]')
plt.ylabel('Mean β PSD [u.a.]')
#plt.legend()
plt.title(f'Dd vs Expected Mean β PSD')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
def sigmoidpulse(x):
    return 0.95 + 1 / (1 + np.exp(-sigm_par * (x - t_mid + neglect_t))) * (0.85 - 0.95) - 1 / (1 + np.exp(-sigm_par * (x - t_mid2 + neglect_t))) * (0.85 - 0.95) 

d_t_attesa = 10*(sigmoidpulse(tt*1000) - 0.85)


%matplotlib widget
window_size = 1  # Sostituisci con la dimensione della finestra desiderata
moving_avg = np.convolve(media, np.ones(window_size)/window_size, mode='same')

# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize = (9,5))

# Plot dei dati originali
ax1.set_yscale('log')

# Aggiungi la moving average al grafico di ax1
ax1.plot(d_t_attesa, moving_avg, color='blue')

# Aggiungi il terzo set di dati con errori associati al grafico esistente
ax1.errorbar(d_t_attesa, media, yerr=error, fmt='o')#, label='simulazioni a Dd costante')

# Imposta altre personalizzazioni per il tuo grafico, se necessario
plt.xlabel('y Dopamine depletion [u.a.]')
plt.ylabel('Mean β PSD [u.a.]')
#plt.legend()
plt.title(f'Dd vs Expected Mean β PSD')
plt.show()

NameError: name 'tt' is not defined